In [39]:
from dotenv import load_dotenv
load_dotenv()
import boto3, json, os
session=boto3.Session(profile_name='stryvia')
polly = session.client("polly")
s3_client=session.client("s3")

In [40]:
def saveToS3(buffer,path):
    response = s3_client.put_object(
        Body=buffer,
        Bucket='ai-tutor-s3',
        Key=path,
    )

In [41]:
def getAudio(speech,key):
    ssml=f"""<speak><prosody rate="120%">{speech}</prosody></speak>"""
    response = polly.synthesize_speech(
    Engine='neural',
    Text=ssml,
    OutputFormat="mp3",
    VoiceId="Matthew",
    TextType="ssml"
    )
    saveToS3(buffer=response["AudioStream"].read(),path=f"voices/{key}.mp3")

In [42]:
def getCaption(speech,key):
    ssml=f"""<speak><prosody rate="120%">{speech}</prosody></speak>"""
    response = polly.synthesize_speech(
    Engine='neural',
    Text=ssml,
    OutputFormat="json",
    VoiceId="Matthew",
    SpeechMarkTypes=["word","sentence"],
    TextType="ssml"
    )
    saveToS3(buffer=response["AudioStream"].read(),path=f"speech_marks/{key}.jsonl")

In [43]:
def saveToLocal(key):
    s3_client.download_file("ai-tutor-s3",f"speech_marks/{key}.jsonl",f"../sample_data/{key}.jsonl")
    s3_client.download_file("ai-tutor-s3",f"voices/{key}.mp3",f"../sample_data/{key}.mp3")

In [44]:
with open(f"../sample_data/{os.environ.get('sentencesKey')}.json",'r') as fp:
    speech=json.load(fp)
key=os.environ.get("testKey")

In [45]:
getAudio(speech,key)


In [ ]:
getCaption(speech,key)

In [ ]:
saveToLocal(key)